In [ ]:
# This notebook is intended to act as an interface for developing plots (both line and residuals) for various weather events.
import pandas as pd
import numpy as np

import numpy.random as random
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime
import pymssql
import os
from datetime import timedelta
import statsmodels.formula.api as sm

import pandas.plotting._converter as pandacnv
pandacnv.register()

from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error

from scipy.stats import iqr

plt.style.use('seaborn')
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

### Legacy Functions
Many of these functions were in use at some point in the research process and have been left as examples.

In [ ]:
def getDB(server, database, user, password):
    conn = pymssql.connect(server=server,
                          database=database,
                          user=user,
                          password=password,
                          login_timeout=300,
                          port=1433)
    cursor = conn.cursor()
    return cursor

In [ ]:
def residualPlot(df, hour='00hr', site="ENTER SITE NAME", out="", save=False):
    date = df.index
    
    try:
        min_date = str(df.index.min().year) + "-" + str(df.index.min().month) + "-" + str(df.index.min().day)
        max_date = str(df.index.max().year) + "-" + str(df.index.max().month) + "-" + str(df.index.max().day)
    except:
        min_date = "NaN"
        max_date = "NaN"
    
    hr = df[hour]
    
    df['colors'] = 'crimson'
    df.loc[hr < 0, 'colors'] = 'dodgerblue'
    
    plt.style.use('seaborn')
    fig, ax = plt.subplots(figsize=(25, 6))#figsize=(11.7, 8.27))
    plt.bar(date, hr, width=0.025, color = df.colors)
    
    # Format the plot
    
    # set title
    title = site + " " + hour + " Sfc. Tmp Residuals (RWIS-HRRR) for " + min_date + " to " + max_date
    plt.title(title)
    
    # set up x-axis
    xticks = plt.xticks(date[::4], rotation=45)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H'))
    plt.xlabel("Datetime: MM-DD HH (EST)")
    
    # set up y-axis
    #plt.ylim((-6, 11))
    yticks = plt.yticks(list(range(-5, 7)))
    plt.ylabel("Residual (Observed - Predicted) in Degrees")
    
    if save:
        plt.savefig("./figs/" + out + "/" + hour + "RWIS_HRRR" + ".png");
    plt.show();

In [ ]:
# Simply checks to see what month the data is coming from to guide what folder to look in for hrrr data
def check_month(month):
    if month == 12:
        return('dec')
    elif month == 1:
        return('jan')
    elif month == 2:
        return('feb')
    elif month == 3:
        return('march')

In [ ]:
def residual_RWIS_NLDAS(df, site="ENTER SITE NAME", out="" , save=False):
    date = df.index.values
    
    try:
        min_date = str(df.index.min().year) + "-" + str(df.index.min().month) + "-" + str(df.index.min().day)
        max_date = str(df.index.max().year) + "-" + str(df.index.max().month) + "-" + str(df.index.max().day)
    except:
        min_date = "NaN"
        max_date = "NaN"
    
    residuals = df.residuals
    
    df['colors'] = 'crimson'
    df.loc[df.residuals < 0, 'colors'] = 'dodgerblue'
    
    plt.style.use('seaborn')
    fig, ax = plt.subplots(figsize=(25, 6))
    plt.bar(date, df.residuals, width=0.025, color = df.colors)
    
    
    # Format the plot
    
    # set title
    title = site + " Residuals (RWIS-NLDAS) for " + min_date + " to " + max_date
    plt.title(title)
    
    # set up x-axis
    xticks = plt.xticks(date[::4], rotation=45)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H'))
    plt.xlabel("Datetime: MM-DD HH (EST)")
    
    # set up y-axis
    yticks = plt.yticks(list(range(-5, 7)))
    plt.ylabel("Residual (Observed - Predicted) in Degrees")
    
    if save:
        plt.savefig("./figs/" + out + "/" + "RWIS_NLDAS " + min_date + " to " + max_date + ".png");
    plt.show();

In [ ]:
def minimize_cases(hour):
    skips = ['00', '01', '02', '03', '04', '05', '07', '08', '09', '10', '11', '13', '14', '15', '16', '17']
    
    if hour in skips:
        return(True)
    else:
        return(False)
    
    

In [ ]:
def lplot(df, rwis, site="ENTER SITE NAME", out="", hour="00", save=False):
    try:
        min_date = str(df.index.min().year) + "-" + str(df.index.min().month) + "-" + str(df.index.min().day)
        max_date = str(df.index.max().year) + "-" + str(df.index.max().month) + "-" + str(df.index.max().day)
    except:
        min_date = "NaN"
        max_date = "NaN"
    
    plt.style.use('seaborn')
    fig, ax = plt.subplots(figsize=(25, 6))
    
    hour = hour + 'hr'
    
    plt.plot(df.index, df.nldas_sfc_tmp, color='dimgrey', label='NLDAS')
    plt.plot(rwis.index, rwis.sfc_tmp, color='steelblue', label='RWIS')
    plt.plot(df.index, df.hrrr_sfc_tmp, color='gold', label='HRRR: ' + hour)
    
    plt.legend(loc='upper left')
    
    # set title
    title = site + ": Surface Temperature in Celcius"
    plt.title(title)
    
    # set up x-axis
    ax.set_xlim([min_date, max_date])
    xticks = plt.xticks(df.index[::4], rotation=45)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H'))
    plt.xlabel("Datetime: MM-DD HH (EST)")
    
    # set up y-axis
    #yticks = plt.yticks(list(range(-10, 8)))
    plt.ylabel("Surface Temperature (Celcius)")
    
    if save:
        plt.savefig("./figs/" + out + "/line/" + hour + " Sfc_tmp Line " + min_date + " to " + max_date + ".png");
    plt.show()

In [ ]:
def scatter_res(df):
    x = dict(df.sum())
    plt.scatter(x.keys(), x.values())

In [ ]:
def lplot_sum_residuals(df, date_df, out="", site="ENTER SITE NAME", save=False):
    data = df.sum()
    
    date_string = str(nldas.index.min()) + ' to ' + str(nldas.index.max())
    
    plt.style.use('seaborn')
    fig, ax = plt.subplots(figsize=(25, 4.5))
    plt.plot(data, '-o', color='dodgerblue', label=date_string)
    plt.legend(loc='best')
    
    # Set title
    title = site + ': Residual Sum of Squares' 
    plt.title(title)
    
    # set up x-axis
    plt.xlabel("Forecast Hour")
    
    # set up y-axis
    plt.ylabel("Sum of Squared Residuals")
    
    if save:
        month_min = str(nldas.index.min().month)
        day_min = str(nldas.index.min().day)
        month_max = str(nldas.index.max().month)
        day_max = str(nldas.index.max().day)
        plt.savefig("./figs/" + out + "/" + site + month_min + "-" + day_min + " to " + month_max + "-" + day_max + ".png");
        
    plt.show()

In [ ]:
def line_plot_sfcTmp(df, site="ENTER SITE NAME", out="", hour="00", save=False):
    try:
        min_date = str(df.index.min().year) + "-" + str(df.index.min().month) + "-" + str(df.index.min().day)
        max_date = str(df.index.max().year) + "-" + str(df.index.max().month) + "-" + str(df.index.max().day)
    except:
        min_date = "NaN"
        max_date = "NaN"
    
    plt.style.use('seaborn')
    fig, ax = plt.subplots(figsize=(25, 6))
    
    hour = hour + 'hr'
    
    plt.plot(df.index, df.nldas_sfc_tmp, color='dimgrey', label='NLDAS')
    plt.plot(df.index, df.rwis_sfc_tmp, color='steelblue', label='RWIS')
    plt.plot(df.index, df.hrrr_sfc_tmp, color='gold', label='HRRR: ' + hour)
    
    plt.legend(loc='upper left')
    
    # set title
    title = site + ": Surface Temperature in Celcius"
    plt.title(title)
    
    # set up x-axis
    xticks = plt.xticks(df.index[::4], rotation=45)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H'))
    plt.xlabel("Datetime: MM-DD HH (EST)")
    
    # set up y-axis
    #yticks = plt.yticks(list(range(-10, 8)))
    plt.ylabel("Surface Temperature (Celcius)")
    
    if save:
        plt.savefig("./figs/" + out + "/line/" + hour + " Sfc_tmp Line " + min_date + " to " + max_date + ".png");
    plt.show()

In [ ]:
def plot_sum_residuals(df):
    colors = ['black', 'red', 'orangered', 'orange', 'brown', 'gold', 'chartreuse', 'darkgreen', 'mediumspringgreen', 'lightseagreen', 'darkcyan', 'darkturquoise', 
              'deepskyblue', 'slategray', 'navy', 'blue', 'darkorchid', 'mediumvioletred', 'crimson']
    data = df.sum()
    
    for key in data.keys():
        x=key
        y=data[key]
        plt.scatter(x,y, color=colors.pop())
    
    plt.legend(test.keys(), loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=5)
        

In [ ]:
def day_night_maePlot(rwis, merged, hour, site="ENTER SITE NAME", out="", lFlag=False, save=False):
    try:
        min_date = str(merged.index.min().month) + "-" + str(merged.index.min().day) + "-" + str(merged.index.min().year)
        max_date = str(merged.index.max().month) + "-" + str(merged.index.max().day) + "-" + str(merged.index.max().year)
    except:
        min_date = "NaN"
        max_date = "NaN"
    
    # Calculate the absolute error
    sunny_abs_res = abs(rwis[rwis.index.isin(merged.index)]['sfc_tmp'] - merged[merged['hrrr_dswrf'] > 5]['hrrr_sfc_tmp'])
    sunny_n = len(sunny_abs_res)
    mae_sunny = sunny_abs_res.sum()/sunny_n
    
#     print(sunny_abs_res.head(10))
#     print('mae_sunny: ' + str(mae_sunny))
    
    night_abs_res = abs(rwis[rwis.index.isin(nldas.index)]['sfc_tmp'] - merged[merged['hrrr_dswrf'] < 5]['hrrr_sfc_tmp'])
    night_n = len(night_abs_res)
    mae_night = night_abs_res.sum()/night_n
    
#     print(night_abs_res.head(10))
#     print(mae_night)
    
    plt.style.use('seaborn')
    
    # set title
    title = site + ': ' + min_date + ' - ' + max_date
    plt.title(title)
    
    # set up x-axis
    plt.xlabel("Forecast Hour")
    
    # set up y-axis
    yticks = plt.yticks(list(np.arange(0, 3.2, .2)))
    ylim =plt.ylim(0,3.2)
    plt.ylabel("Mean Absolute Error")
    
    # Only plot labels if True
    if lFlag:
        plt.scatter(hour+'hr', mae_sunny, label='Night Time Residuals', color='navy')
        plt.scatter(hour+'hr', mae_night, label='Day Time Residuals', color='gold')
        plt.legend(loc='best')
    else:
        plt.scatter(hour+'hr', mae_sunny, color='navy')
        plt.scatter(hour+'hr', mae_night, color='gold')
    
    if save:
        month_min = str(merged.index.min().month)
        day_min = str(merged.index.min().day)
        month_max = str(merged.index.max().month)
        day_max = str(merged.index.max().day)
        plt.savefig("./figs/" + 'DayNight_MAE-' + month_min + "-" + day_min + " to " + month_max + "-" + day_max + ".png");
        
#day_night_maePlot(rwis, merged, hour=hrrr_file_hr, site=sql_site, out=site, lFlag=labelFlag, save=False)

### Analysis and plotting begins

In [ ]:
# below min and max are for the entire 4 months
min_event_time = pd.to_datetime('2018-12-01 00:00:00')
max_event_time = pd.to_datetime('2019-03-31 23:00:00')

time_range = pd.date_range(start=min_event_time, end=max_event_time, freq='1min')

### Select Your Site
It's important here to comment/uncomment the site you do or don't want. It's currently set up to only handle 1 at a time.

### Read from pickles (alternative to sql calls)

In [ ]:
rwis = pd.read_pickle('./pickledData_sql/ftWayne.pkl')
# rwis = pd.read_pickle('./pickledData_sql/gasCity.pkl')
# rwis = pd.read_pickle('./pickledData_sql/i74@i465.pkl')
# rwis = pd.read_pickle('./pickledData_sql/us31@sr38.pkl')
# rwis = pd.read_pickle('./pickledData_sql/jeffersonville.pkl')
# rwis = pd.read_pickle('./pickledData_sql/scottsburg.pkl')

# Minor processing of the data
rwis['sfc_tmp'] = pd.to_numeric(rwis['sfc_tmp'], errors='coerce')
rwis['air_tmp'] = pd.to_numeric(rwis['air_tmp'], errors='coerce')
rwis['dewpt'] = pd.to_numeric(rwis['dewpt'], errors='coerce')
rwis.tstamp = pd.to_datetime(rwis.tstamp)
rwis = rwis.set_index('tstamp')

# Convert tmps from F to C
rwis.sfc_tmp = (rwis.sfc_tmp - 32) * (5/9)
rwis.air_tmp = (rwis.air_tmp - 32) * (5/9)
rwis.dewpt = (rwis.dewpt - 32) * (5/9)

# There is missing data in the rwis, reindex it to be a full data set
rwis = rwis.reindex(pd.date_range(start=rwis.index.min(), end=rwis.index.max(), freq='1min'))

In [ ]:
maes1 = []
maes2 = []
maes3 = []
maes4 = []
maes5 = []

residuals1 = pd.DataFrame()
residuals2 = pd.DataFrame()
residuals3 = pd.DataFrame()
residuals4 = pd.DataFrame()
residuals5 = pd.DataFrame()

# store select hrrr data
hrrr0=[]
hrrr6=[]
hrrr12=[]
hrrr18=[]

In [ ]:
def process_residuals_mae(combination, maes, residuals):
    tmp = combination[['sfc_tmp', 'hrrr_sfc_tmp']].dropna()
    maes.append(mean_absolute_error(tmp['sfc_tmp'], tmp['hrrr_sfc_tmp']))

    # calculate the residuals
    residuals[hrrr_file_hr + 'hr'] = tmp['sfc_tmp'] - tmp['hrrr_sfc_tmp'] 
    
    return(maes, residuals)

In [ ]:
# Need the HRRR data now, this will have to be a loop in order to get all of the data.
# The residuals will need to be calculated in this loop.

hrrr_path = './hrrr_closest2rwis_processed/all_out/fort_wayne/'
#hrrr_path = './hrrr_closest2rwis_processed/all_out/gas_city/'
#hrrr_path = './hrrr_closest2rwis_processed/all_out/i74@i465/'
#hrrr_path = './hrrr_closest2rwis_processed/all_out/us31@sr38/'
#hrrr_path = './hrrr_closest2rwis_processed/all_out/jeffersonville/'
#hrrr_path = './hrrr_closest2rwis_processed/all_out/scottsburg/'
hrrr_files = os.listdir(hrrr_path)

morning_dswrf = []
evening_dswrf = []

count = 0 # tells where to insert nan into the maes list if not enough data is present for the calculation

res_vars = [residuals1, residuals2, residuals3, residuals4, residuals5]
maes_vars = [maes1, maes2, maes3, maes4, maes5]

loop_count = 0
for residuals, maes in zip(res_vars, maes_vars):
    loop_count += 1
    count = 0 # tells where to insert nan into the maes list if not enough data is present for the calculation
    print("Loop: " + str(loop_count))
    for file in hrrr_files:
        hrrr_file_hr = file[-6:-4]
        
        
#         file='dec-march18.csv' # RWIS INDICATED LOCK IN FILE
        
        
        hrrr = pd.read_csv(hrrr_path + file, parse_dates=['raw_timestamp', 'timestamp_fcst', 'timestamp_est'])#, index_col=['timestamp_est'])
        hrrr['timestamp_est'] = pd.to_datetime(hrrr['timestamp_est'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        hrrr = hrrr.set_index(hrrr['timestamp_est'], drop=True)

        hrrr = hrrr.loc[~hrrr.index.duplicated(keep='first')] # Ensure unique index. 02 hour data has an issue with duplicates.
        mask = (hrrr.index >= min_event_time) & (hrrr.index <= max_event_time) # For subsetting the data to a specified time window
        hrrr = hrrr.loc[mask]

        # Collect column names for filtering
        hrrr_sfc_tmp = [col for col in hrrr.columns if 'TMP_surface' in col][0] # Find the surface temperature column for hrrr
        hrrr_dswrf = [col for col in hrrr.columns if 'DSWRF' in col][0] 
        hrrr_dlwrf = [col for col in hrrr.columns if 'DLWRF' in col][0]
        hrrr_2m_tmp = [col for col in hrrr.columns if 'TMP_2 m' in col][0] 
        hrrr_dpt = [col for col in hrrr.columns if 'DPT_2 m' in col][0] 
        hrrr_rh = [col for col in hrrr.columns if 'RH_2 m' in col][0]
        hrrr_pot = [col for col in hrrr.columns if 'POT_2 m' in col][0]
        hrrr_shtfl = [col for col in hrrr.columns if 'SHTFL_surface' in col][0]
        hrrr_gflux = [col for col in hrrr.columns if 'GFLUX_surface' in col][0]   
        
#         hrrr = hrrr[(hrrr[hrrr_2m_tmp] - hrrr[hrrr_dpt]) > 1.5] ############ EXPERIMENTAL HRRR FILTER
        
        # Some of the data has the wrong typing
        hrrr[hrrr_sfc_tmp] = pd.to_numeric(hrrr[hrrr_sfc_tmp], errors='coerce')
        hrrr[hrrr_dswrf] = pd.to_numeric(hrrr[hrrr_dswrf], errors='coerce')
        

        # combine all of the relevant data into one dataframe with the correct time bounds
        combination = pd.DataFrame(index = time_range) # container for data aggregation

        combination['hrrr_sfc_tmp'] = hrrr.loc[hrrr.index.isin(combination.index)][hrrr_sfc_tmp]
        combination['hrrr_dswrf'] = hrrr.loc[hrrr.index.isin(combination.index)][hrrr_dswrf]
        combination = combination.join(rwis, how='outer')

        print(file)

        # Determine the solar radiation threshold. 0's are removed to prevent the solar radiation threshold from being drug down.
        tmp = hrrr.between_time('07:00:00', '10:00:00')
        tmp = tmp[tmp[hrrr_dswrf] > 0]
        #morning_dswrf.append(sum(tmp[hrrr_dswrf])/len(tmp[hrrr_dswrf]))
        tmp = hrrr.between_time('16:00:00', '19:00:00')    
        tmp = tmp[tmp[hrrr_dswrf] > 0]
        #evening_dswrf.append(sum(tmp[hrrr_dswrf])/len(tmp[hrrr_dswrf]))
        
# # RWIS INDICATED LOCK IN FILE; To generate the RWIS indicated figures, break the loop here and comment everything below.
#         break
#     break


        if hrrr_file_hr == '00':
            hrrr0 = combination
        if hrrr_file_hr == '06':
            hrrr6 = combination
        if hrrr_file_hr == '12':
            hrrr12 = combination
        if hrrr_file_hr == '18':
            hrrr18 = combination
            
            
        # Filter Data on RWIS based measurements
        
        #No filter Case
        if loop_count == 1:
            # Check if the subset results in 1 or fewer data points.
            if combination.shape[0] < 1:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
            
            try:                
                maes, residuals = process_residuals_mae(combination, maes, residuals)
                
            except:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
            
        if loop_count == 2:
            combination = combination[combination['hrrr_dswrf'] > 170] # above solar radiation threshold
            # Check if the subset results in 1 or fewer data points.
            if combination.shape[0] < 1:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
            try:                
                maes, residuals = process_residuals_mae(combination, maes, residuals)
                
            except:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
            
        if loop_count == 3:
            combination = combination[combination['hrrr_dswrf'] <= 170] # below solar radiation threshold
            # Check if the subset results in 1 or fewer data points.
            if combination.shape[0] < 1:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
            
            try:
                maes, residuals = process_residuals_mae(combination, maes, residuals)
                
            except:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
                

                
        if loop_count == 4:
            combination = combination[(combination['air_tmp'] - combination['dewpt']) <= 1.5] # Near saturation filter
            # Check if the subset results in 1 or fewer data points.
            if combination.shape[0] < 1:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
            
            try:
                maes, residuals = process_residuals_mae(combination, maes, residuals)
            except:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue
            
            
        if loop_count == 5:
            combination = combination[combination['precip'] == 1] # YES Precipitation
            # Check if the subset results in 1 or fewer data points.
            if combination.shape[0] < 1:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue           
   
            try:
                maes, residuals = process_residuals_mae(combination, maes, residuals)
            
            except:
                print("Not enough data fits that criteria.")
                maes.insert(count, np.nan)
                continue       

        count += 1
#     ONLY GET THE NO FILTER SCENARIO; enable the break below to ensure only the no filter is captured
#     break

### Find the sample size for each residual.

In [ ]:
count = 1
for var in res_vars:
    print('Showing residuals' + str(count))
    for col in var:
        print(len(var[col].dropna()))
    count += 1


#### To capture the MAE values for no filters 1, 2, 3, and 4 at different time slices ...
- You'll need to run each of these once after adjusting the time window

The process looks like this
- set the time
- run the code
- run maes_nf1
- reduce the time window
- run the code
- run maes_nf2
- continue on like this until done

Note: To do this, you have to enable the break that is commented at the end of the code.

In [ ]:
maes_nf1 = []
maes_nf2 = []
maes_nf3 = []

In [ ]:
# save the no filter across each time slice
maes_nf1 = maes1
maes_nf2 = maes1
maes_nf3 = maes1

In [ ]:
# This set of plots is for dialing in and showing only the no filter scencario across various sets
fig = plt.plot(maes_nf1, '-o', label='Full Storm: 24 Hours', color='darkblue', alpha=0.7, markersize=4, linewidth=1)
fig = plt.plot(maes_nf2, '-o', label='Core: 11:00 a.m. - 6:00 p.m.', color='goldenrod', alpha=0.7, markersize=4, linewidth=1)
fig = plt.plot(maes_nf3, '-o', label='Single Hour: 3:00 p.m.', color='firebrick', alpha=0.7, markersize=4, linewidth=1)

xticks = plt.xticks(range(0,19))
plt.xlabel("Forecast Hour", fontsize=10)

yticks = plt.yticks(np.arange(0,6.01,1)) # originally 0 to 6.01 by 0.2
#ylim =plt.ylim(2,4.25)
plt.ylabel("MAE Score in Degrees Celcius", fontsize=10)

plt.title('US-31 at SR-38', fontsize=10)

plt.legend(loc='best', frameon=False)

fig = plt.gcf()
plt.style.use('default')
fig.set_size_inches(16, 9)
# plt.style.use('seaborn')
# fig.set_size_inches(3.5, 3)

ax=plt.gca()
n = 2
for index, label in enumerate(ax.xaxis.get_ticklabels()):
    if index % n != 0:
        label.set_visible(False)

plt.savefig('./figs/maeLinePlot_timeBased.png', bbox_inches="tight", dpi=300)

### RWIS Indicated Variables Vs. HRRR Sfc. Tmp. Forecasts
Nothing special needs to be done to run this figure. 
- NOTE: This figures output will not be very clean or useful when ran for four months, subset the data temporally to get better results.

In [ ]:
#Plot the HRRR select forecasts (hrrr0, 6, 12, and 18) against rwis
# fig, axes = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(3.5,3))
fig, axes = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(16,9))
plt.tight_layout()
plt.style.use('seaborn')
# plt.style.use('default')
mask = (rwis.index >= min_event_time) & (rwis.index <= max_event_time) # For subsetting the data to a specified time window
rwis = rwis.loc[mask]

#Locate precipiation
# tmp = rwis.dropna()['precip'] == 1
# tmp = tmp[tmp == True]
# tmp = tmp.replace(True, 0)

plt.plot(rwis.dropna()['sfc_tmp'], color='black', linewidth=1, label='Surface Temperature')
plt.plot(rwis.dropna()['air_tmp'], color='orangered', linewidth=1, label='Air Temperature')
plt.plot(rwis.dropna()['dewpt'], color='springgreen', linewidth=1, label='Dewpoint Temperature')
#plt.plot(tmp, 'o', color='forestgreen', label='Precipitation', alpha=0.3)

plt.plot(hrrr0['hrrr_sfc_tmp'].dropna(), 'o', markersize=3, color='royalblue', label='Current')
plt.plot(hrrr6['hrrr_sfc_tmp'].dropna(), 's', markersize=3, color='mediumvioletred', label='6 Hours Ago')
plt.plot(hrrr12['hrrr_sfc_tmp'].dropna(), 'D', markersize=3, color='seagreen', label='12 Hours Ago')
plt.plot(hrrr18['hrrr_sfc_tmp'].dropna(), '^', markersize=3, color='darkorange', label='18 Hours Ago')

tmp = hrrr6.dropna()['hrrr_dswrf'] > 170
tmp = tmp[tmp == True]
tmp = tmp.replace(True, 0)

axes.bar(tmp.index, height=20, bottom=-10, align='edge', width=0.039, color='gold', alpha=0.3, label='DSWRF > 170: 6 hrs ago')
axes.set_ylim(-7,7)

plt.title('Fort Wayne', fontsize=10);
plt.xlabel("Hour (EST)", fontsize=10);
#xticks = plt.xticks(rotation=10)
plt.ylabel('Degrees Celcius', fontsize=10)
yticks = plt.yticks(np.arange(-7,8,2))
plt.legend(loc='best', frameon=False);

axes.xaxis.set_major_formatter(mdates.DateFormatter('%H'))

n = 2
for index, label in enumerate(axes.xaxis.get_ticklabels()):
    if index % n != 0:
        label.set_visible(False)

plt.axhline(y=0, color='r', linewidth=0.5)
        
# axes.xaxis.grid(False)
# for label in axes.xaxis.get_ticklabels()[::2]:
#     label.set_visible(False)

plt.savefig('./figs/4month_rwis_indicated.png', bbox_inches='tight', dpi=300)

### MAE Line plot showing differing filters MAE values for each forecast hour.

In [ ]:
# plt.style.use('default')
plt.style.use('seaborn')
fig = plt.plot(maes1, '-o', label='No Filter', color='black', alpha=0.7, markersize=4, linewidth=1)
fig = plt.plot(maes2, '-o', label='DSWRF > 170', color='orange', alpha=0.7, markersize=4, linewidth=1)
fig = plt.plot(maes3, '-o', label='DSWRF <= 170', color='mediumorchid', alpha=0.7, markersize=4, linewidth=1)
fig = plt.plot(maes4, '-o', label='TMP - DPT within 1.5C', color='mediumturquoise', alpha=0.7, markersize=4, linewidth=1)
fig = plt.plot(maes5, '-o', label='Precip.', color='forestgreen', alpha=0.7, markersize=4, linewidth=1)

xticks = plt.xticks(range(0,19))
plt.xlabel("Forecast Hour", fontsize=10)

yticks = plt.yticks(np.arange(0,5.01,1)) # originally 0 to 6.01 by 0.2
#ylim =plt.ylim(2,4.25)
plt.ylabel("MAE Score in Degrees Celcius", fontsize=10)

#plt.title('MAE Score of Individual Forecast Hours', fontsize=10)
plt.title('Fort Wayne', fontsize=10)

plt.legend(loc='best', frameon=False)

fig = plt.gcf()
fig.set_size_inches(16, 9)
# fig.set_size_inches(3.5, 3)
# fig.set_size_inches(3.6, 3.5)

ax=plt.gca()
n = 2
for index, label in enumerate(ax.xaxis.get_ticklabels()):
    if index % n != 0:
        label.set_visible(False)

plt.savefig('./figs/4month_mae.png', bbox_inches="tight", dpi=300)

### View the CDF for your residuals across forecast hours for the no filter case.

In [ ]:
cmap = sns.color_palette("Spectral", 19)

for color, hour in zip(cmap, residuals.columns):
    try:
        if len(residuals1[hour].dropna()) >= 2:
            fig = sns.kdeplot(residuals1[hour].dropna(), cumulative=True, color=color)
            print(hour + " length is " + str(len(residuals1[hour].dropna())))
        else:
            # ensures that the legend entries will always remain the same. The values are intended to be
            # absurd so that it won't interfere with the real plot.
            print('plot dummy')
            print(hour + " length is " + str(len(residuals1[hour].dropna())))
            fig = sns.kdeplot([1000, 1005, 1012, 1104], cumulative=True, color=color, label=hour)
            next()
    except:
        print('Conditions unmet for ' + hour)

fig.set_xlim(-6, 6);
plt.xlabel('Residuals of Temperature in Celcius');
plt.title("Cumulative Distribution Function for Individual Forecast Hours")
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

plt.savefig('./figs/4month_cdf.png', bbox_inches="tight", dpi=300)

### Boxplots
The following two cells are the process for creating the box plots for this analysis. Residuals are coerced into a format conducive to plotting in the first cell and then plotted in the second.

In [ ]:
# Add appropriate suffixes for each filter
residuals1 = residuals1.add_suffix(' No Filter')
residuals2 = residuals2.add_suffix(' DSWRF > 170')
residuals3 = residuals3.add_suffix(' DSWRF <= 170')
residuals4 = residuals4.add_suffix(' TMP - DPT <= 1.5C')
residuals5 = residuals5.add_suffix(' Precip.')

# Melt the residuals into a form convenient for plotting
residuals1 = pd.melt(residuals1)
residuals2 = pd.melt(residuals2)
residuals3 = pd.melt(residuals3)
residuals4 = pd.melt(residuals4)
residuals5 = pd.melt(residuals5)

In [ ]:
#f, axes = plt.subplots(1, 19, sharex=False, sharey='row', figsize=(16, 4))
f, axes = plt.subplots(1, 4, sharex=False, sharey='row', figsize=(16, 4))
#f, axes = plt.subplots(1, 2, sharex=False, sharey='row', figsize=(4, 4))
plt.tight_layout()

# Need to track both row and column (in that order)
ax_col = 0
count = 0

# Loop through the unique variables and plot on each subplot
#hours = ['00hr', '01hr', '02hr', '03hr', '04hr', '05hr', '06hr', '07hr', '08hr', '09hr', '10hr', '11hr', '12hr', '13hr', '14hr', '15hr', '16hr', '17hr', '18hr']
hours = ['00hr', '06hr', '12hr', '18hr']
#hours = ['06hr']
for hour in hours:
    
    tmp_df = pd.DataFrame()
    tmp = residuals1[residuals1['variable'].str.contains(hour)]
    tmp2 = residuals2[residuals2['variable'].str.contains(hour)]
    tmp3 = residuals3[residuals3['variable'].str.contains(hour)]
    tmp4 = residuals4[residuals4['variable'].str.contains(hour)]
    tmp5 = residuals5[residuals5['variable'].str.contains(hour)]
    tmp_df = pd.concat([tmp, tmp2, tmp3, tmp4, tmp5]) # tmp7])
    
    print(tmp_df.groupby('variable').median())

    bp = sns.boxplot(x="variable", y="value", data=tmp_df, ax=axes[ax_col], width=0.4, whis='range') #Whiskers show max and min)
    
    # Dynamically build the color list based on data availability
    colors = []
    if len(tmp) > 0:
        colors.append('steelblue')
    if len(tmp2) > 0:
        colors.append('orange')
    if len(tmp3) > 0:
        colors.append('mediumorchid')
    if len(tmp4) > 0:
        colors.append('mediumturquoise')
    if len(tmp5) > 0:
        colors.append('forestgreen')

    for box, color in zip(axes[ax_col].artists, colors):
        box.set_facecolor(color=color)

    if ax_col == 0:
        axes[ax_col].set_xlabel(hour + 's ago')
#         axes[ax_col].set_xlabel('Current')
    else:
        axes[ax_col].set_xlabel(hour + 's ago')
        
    axes[ax_col].set_xticklabels([])
    
    axes[ax_col].set_ylabel('')
    axes[ax_col].set_yticks(np.arange(-8, 9, 1))
        
    ax_col += 1
    count += 1
    

if len(hours) <= 4:
    out_cols = 4
else:
    out_cols = 19
    
for ax_col in range(0, out_cols):
    # Set up the y-axis
    #axes[ax_col].set_yticks(np.arange(-8, 9, 2));
    axes[ax_col].set_yticks(np.arange(-8, 22, 2));
    axes[ax_col].get_ygridlines()[4].set_color('red')

axes[0].set_ylabel('Residuals in Degrees Celcius')
plt.ylim(-8,20)

plt.savefig('./figs/4month_boxplot.png', bbox_inches="tight", dpi=300)